In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision as vision
from torchvision.datasets import MNIST
from torchvision.utils import make_grid

In [28]:
data=MNIST(root="data",download=True,transform=vision.transforms.ToTensor())
data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

# Model

In [29]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [30]:
class feedf(nn.Module):
    def __init__(self,ip_size,h_size,op_size):
        super().__init__()
        self.linear1=nn.Linear(ip_size,h_size)
        self.linear2=nn.Linear(h_size,op_size)
    
    def forward(self,xb):
        xb=xb.view(xb.size(0),-1)
        out=self.linear1(xb)
        out=nn.functional.relu(out)
        out=self.linear2(out)
        return out
    
    def training_step(self,batch):
        img,l=batch
        pred=self(img)
        loss=nn.functional.cross_entropy(pred,l)
        return loss
    
    def validation_step(self,batch):
        img,l=batch
        pred=self(img)
        loss=nn.functional.cross_entropy(pred,l)
        acc=accuracy(pred,l)
        return {"val_loss":loss,"val_acc":acc}
    
    def validation_epoch_end(self,output):
        loss=[x["val_loss"] for x in output]      
        acc=[x["val_acc"] for x in output]      
        loss=torch.stack(loss).mean()
        acc=torch.stack(acc).mean()
        return {"loss":loss.item(),"acc":acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    
            
            
        

In [31]:
ts,vs=torch.utils.data.random_split(data,[50000,10000])
tl=DataLoader(ts,batch_size=128,pin_memory=True,num_workers=2)
vl=DataLoader(vs,batch_size=128,pin_memory=True,num_workers=2)

In [32]:
model=feedf(784,32,10)

In [33]:
for i,j in tl:
    pred=model(i)
    loss=nn.functional.cross_entropy(pred,j)
    print(loss)
    break
    

tensor(2.3089, grad_fn=<NllLossBackward>)


# GPU

In [34]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

In [35]:
def to_device(data,device):
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device)

In [36]:
class DeviceDL():
    def __init__(self,dl,device):
        self.dl=dl
        self.device=device
    def __iter__(self):
        for i in self.dl:
            yield to_device(i,self.device)
    def __len__(self):
        return len(self.dl)

In [37]:
dev=get_device()
tl=DeviceDL(tl,dev)
vl=DeviceDL(vl,dev)

In [38]:
for i,j in tl:
    print(i)
    print(j)
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [39]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epoch,lr,model,train_loader,valid_loader):
    history=[]
    optim=torch.optim.SGD(model.parameters(),lr=lr)
    for i in range(epoch):
        for i in train_loader:
            loss=model.training_step(i)
            loss.backward()
            optim.step()
            optim.zero_grad()
        result=evaluate(model,valid_loader)
#         model.epoch_end(epoch,result)
        history.append(result)
    return history
            
        

In [40]:
dev=get_device()
model=feedf(784,32,10)
to_device(model,dev)

feedf(
  (linear1): Linear(in_features=784, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=10, bias=True)
)

In [42]:
fit(10,0.05,model,tl,vl)

[{'loss': 0.2699311673641205, 'acc': 0.9234572649002075},
 {'loss': 0.25744953751564026, 'acc': 0.9270173907279968},
 {'loss': 0.2470676153898239, 'acc': 0.9309731125831604},
 {'loss': 0.23835764825344086, 'acc': 0.9324564933776855},
 {'loss': 0.23059599101543427, 'acc': 0.9347310066223145},
 {'loss': 0.22361010313034058, 'acc': 0.9359177350997925},
 {'loss': 0.2173796147108078, 'acc': 0.9371044039726257},
 {'loss': 0.2115447223186493, 'acc': 0.9378955960273743},
 {'loss': 0.20620688796043396, 'acc': 0.9393789768218994},
 {'loss': 0.2012401521205902, 'acc': 0.9408623576164246}]

In [43]:
for i in model.parameters():
    print(i.device)

<generator object Module.parameters at 0x7f337412eb50>